In [1]:
import pandas as pd

# Download Covid Data for Chapter XX from Sources, Downselect, and Combine

This notebook will download the data used in the Covid example in Chapter XX from the sources (with one exception), do the data down-selection, rename columns, combine into one dataframe, and write that dataframe out as a single CSV file. 

*(For those who are more familiar with Python and Pandas, a Pickle file would be easier to work with, but I want the reader of the book to gain experience in reading a CSV file into a Pandas dataframe and working with that)*

## Cumulative Number of Covid Cases by Date from New York Times

In [2]:
# Download the Covid Data from the NYT github. Use the Raw URL to directly retrieve the data:
df=pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv")

In [3]:
# Now restrict to just April 30, 2020
df=df[df["date"]=="2020-04-30"]

In [4]:
# Index by state so that we can easily merge with other data sets
df=df.set_index("state")

In [5]:
# Drop the data we won't need and limit to states
df.drop(["date", "fips","deaths"], axis=1, inplace=True)
df.drop(["District of Columbia", "Guam", "Northern Mariana Islands", "Puerto Rico", "Virgin Islands"],
       inplace=True)

## Population Data from the US Census Bureau

In [6]:
# Download population data from the US Census Bureau
# You will have to have the 'opynpyxl' library installed because this is an XLSX file
population = pd.read_excel(
     "https://www2.census.gov/programs-surveys/popest/tables/2010-2019/state/totals/nst-est2019-01.xlsx",
     skiprows=[0,1,2,4,5,6,7,8,60,61,62,63,64,65,66,67,68],
    engine='openpyxl'
)

In [7]:
# Rename the columns we care about
population.rename(columns={'Unnamed: 0':'state'}, inplace=True)

In [8]:
# Drop the columns we won't need
cols=population.columns
cols=cols.drop('state')
cols=cols.drop(2019)
print(cols)
population.drop(cols, axis=1, inplace=True)

Index([        'Census', 'Estimates Base',             2010,             2011,
                   2012,             2013,             2014,             2015,
                   2016,             2017,             2018],
      dtype='object')


In [9]:
# Remove the stray periods in the state names and make state the index
population["state"]=population["state"].str.replace(".","")
population.set_index("state", inplace=True)

In [10]:
# Combine the population data into our main data frame
df["population"]=population[2019]

## GDP Data from the US Department of Commerce Bureau of Economic Analysis

In [12]:
# The GDP data cannot be easily directly downloaded because it relies on JavaScript.
# Instead, I am going to provide a file, and you can substitute the CSV file from
# downloading from
gdp_url="https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=30&isuri=1&year_end=-1&download=csv&classification=non-industry&state=0&yearbegin=-1&unit_of_measure=levels&major_area=0&area=xx&year=2019&tableid=532&category=5532&area_type=0&statistic=3"
gdp=pd.read_csv("https://raw.githubusercontent.com/jmshea/Foundations-of-Data-Science-with-Python/main/03-first-data/state-gdp-bea.csv",
                skiprows=[0, 1, 2, 3, 5, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66])

In [13]:
# Drop columns we don't care about
gdp.drop(["GeoFips", "2019:Q1", "2019:Q2", "2019:Q3"], axis=1, inplace=True)

In [14]:
# Now rename the state column and set the index
gdp.rename(columns={"GeoName":"state"}, inplace=True)
gdp.set_index("state", inplace=True)


In [15]:
# Add the GDP data to our main data frame:
df["gdp"]=gdp["2019:Q4"]

## Urban/Rural Data from the US Census Bureau

In [16]:
# Download the data about urban populations from the US Census Bureau
urban=pd.read_excel("https://www2.census.gov/geo/docs/reference/ua/PctUrbanRural_State.xls")

In [17]:
# Drop the columns we don't care about
cols=urban.columns
cols=cols.drop('STATENAME')
cols=cols.drop('POPPCT_URBAN')
urban.drop(cols, axis=1, inplace=True)

In [18]:
# Rename the state columns to match our main dataframe and make it the index:
urban.rename(columns={"STATENAME":"state"}, inplace=True)
urban.set_index("state", inplace=True)

In [19]:
# Add the % urban data to our main data frame:
df["urban"]=urban["POPPCT_URBAN"]

In [20]:
# Output the main data frame to the screen to make sure it looks right
df

,cases,population,gdp,urban
state,,,,
Alabama,7068,4903185,230750.1,59.04
Alaska,353,731545,54674.7,66.02
Arizona,7648,7278717,379018.8,89.81
Arkansas,3281,3017804,132596.4,56.16
California,50470,39512223,3205000.1,94.95
Colorado,15207,5758736,400863.4,86.15
Connecticut,27700,3565287,290703.0,87.99
Delaware,4734,973764,77879.4,83.30
Florida,33683,21477737,1126510.3,91.16


## Write the Data Back to CSV

In [21]:
# Here I write the data out to a CSV file so that the reader gains experience
# reading in and working with data frames created from CSV file

In [22]:
df.to_csv("covid-merged.csv")